In [17]:
import pandas as pd
import numpy as np
import awoc
from sklearn import preprocessing

In [18]:
df = pd.read_csv("../LifeExpectancyData.csv")

In [19]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [20]:
columns_to_standardize = list(df.columns)[3:]

Here we do normalization.

We could maybe also do min-max normalization to get values between 0 and 1

In [21]:
for column in columns_to_standardize:
    mean = df[column].mean()
    std_dev = df[column].std()
    df[column] = (df[column] - mean) / std_dev

In [22]:
df.head(5)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,-0.443615,0.790103,0.268778,-1.133365,-0.335513,-0.635838,-0.110366,...,-3.267459,0.889322,-0.730453,-0.32339,-0.483449,0.343917,2.796324,2.756711,-0.704355,-0.563512
1,Afghanistan,2014,Developing,-0.979112,0.854468,0.285738,-1.133365,-0.334384,-0.755503,-0.168095,...,-1.047897,0.897327,-0.856945,-0.32339,-0.481456,-0.203661,2.864194,2.801068,-0.718580,-0.593284
2,Afghanistan,2013,Developing,-0.979112,0.830331,0.302697,-1.133365,-0.334537,-0.675726,-0.173502,...,-0.877162,0.877314,-0.772617,-0.32339,-0.480121,0.311058,2.909441,2.845424,-0.747029,-0.623055
3,Afghanistan,2012,Developing,-1.021112,0.862513,0.328137,-1.133365,-0.332040,-0.556061,0.032040,...,-0.663742,1.033419,-0.646125,-0.32339,-0.477443,-0.148436,2.954688,2.911960,-0.780219,-0.652827
4,Afghanistan,2011,Developing,-1.052611,0.886650,0.345097,-1.133365,-0.367800,-0.516173,0.051748,...,-0.621059,0.773244,-0.603961,-0.32339,-0.519939,-0.160210,3.022558,2.956317,-0.822893,-0.742141


In [23]:
df[df.isna().any(axis=1)].head(5)

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
32,Algeria,2015,Developing,0.669378,-1.173015,-0.078896,NaN,-0.371370,0.560811,-0.205506,...,0.531406,NaN,0.534467,-0.32339,-0.234783,0.444472,0.262499,0.206189,0.547402,0.716661
44,Algeria,2003,Developing,0.259881,-0.151228,-0.087376,-1.051932,-0.358784,NaN,1.129685,...,0.189935,-0.935905,0.197155,-0.32339,-0.503766,-0.155868,0.330369,0.272725,0.168081,-0.146712
45,Algeria,2002,Developing,0.249381,-0.159274,-0.087376,-1.046996,-0.296662,NaN,0.300194,...,0.147251,-0.883870,0.154991,-0.32339,-0.400053,-0.156589,0.330369,0.294903,0.120666,-0.265798
46,Algeria,2001,Developing,0.228381,-0.159274,-0.087376,-1.079076,-0.296927,NaN,0.023232,...,0.275303,-0.839840,0.281483,-0.32339,-0.402960,0.308771,0.352993,0.317082,0.077993,-0.325341
47,Algeria,2000,Developing,0.217881,-0.159274,-0.078896,-1.074141,-0.293672,NaN,-0.211000,...,0.147251,-0.979934,0.154991,-0.32339,-0.401255,-0.157920,0.375616,0.339260,0.040061,-0.384884


Remove all nans and save csv

In [12]:
# df = df.dropna()
# df.to_csv("normalizedData-RemovedNanRows.csv")
# df

This removes about 50% of about data - we need to be more careful about removing these nans...

Fill in Nan values based on this approach: 

If there is other values from the same country, we take the mean of those values and fill in empty data. (if there is none, we use mean) 

If over 200 nans in one column, we remove as they have way too many nans

In [24]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [25]:
df.drop(["Hepatitis B", "GDP", "Population"], axis="columns", inplace=True)

In [26]:
columns_to_correct_nans = list(df.columns)[3:]

In [27]:
start_nans = df.isna().sum().sum()
df_without_nans = df.copy()
print("Number of nans before this procedure: ", start_nans)
for index, row in df_without_nans.iterrows():
    for column in columns_to_correct_nans:
        if pd.isna(row[column]):
            current_country = row['Country']
            same_country_rows = df_without_nans[df_without_nans['Country'] == current_country]
            mean_value = same_country_rows[column].mean()
            df_without_nans.at[index, column] = mean_value

print("Filled with mean from rows with same country: ", start_nans - df_without_nans.isna().sum().sum())
start_nans = df_without_nans.isna().sum().sum()
df_without_nans.fillna(df_without_nans.mean(), inplace=True)
print("Filled with mean from all rows: ", start_nans - df_without_nans.isna().sum().sum())

Number of nans before this procedure:  910
Filled with mean from rows with same country:  409
Filled with mean from all rows:  501


/var/folders/pl/7f1cr2657p3bnpdt3bnm_9w00000gn/T/ipykernel_19124/3355308998.py:14: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_without_nans.fillna(df_without_nans.mean(), inplace=True)


In [28]:
df_without_nans.shape, df_without_nans.isna().sum().sum()

((2938, 19), 0)

In [29]:
df_without_nans.to_csv("StandardizedDataFrameWithNansFilled.csv")